In [3]:
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()  # Assurez-vous que le chemin vers .env est correct si nécessaire

# Utiliser les variables d'environnement
server = os.getenv('DB_SERVER')
database = os.getenv('DB_NAME')
username = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
driver = os.getenv('DB_DRIVER')

# Définir la chaîne de connexion
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"  # Nécessaire pour Azure
    f"TrustServerCertificate=no;"  # Nécessaire pour Azure
    f"Connection Timeout=30;"
)

# Lire le fichier CSV
df = pd.read_csv('data/movies_data.csv')

In [4]:
# Nom du schéma
schema_name = 'appmovieschema'  # Remplacez par le nom de votre schéma

# Établir la connexion
try:
    with pyodbc.connect(conn_str) as conn:
        print("Connexion réussie.")
        cursor = conn.cursor()

        # Extraire les genres uniques
        genres = set()
        for genre_list in df['genres'].str.split('|').dropna():
            genres.update(genre_list)

        # Insérer les genres uniques dans Genre_Table avec le schéma spécifié
        for genre_id, genre in enumerate(genres, start=1):
            cursor.execute(f"IF NOT EXISTS (SELECT 1 FROM {schema_name}.Genre_Table WHERE genre_name = ?) INSERT INTO {schema_name}.Genre_Table (genre_id, genre_name) VALUES (?, ?)", genre, genre_id, genre)

        conn.commit()

        # Pour chaque film, attribuer un 'movie_id' et insérer ses genres dans MovieGenre_Table
        for index, row in df.iterrows():
            movie_title = row['title']
            genres_data = row['genres']
            release_date = row['release_date']
            poster_link = row['poster_link']

            # Vérifier si genres_data est une chaîne de caractères (str) avant de la diviser
            if isinstance(genres_data, str):
                movie_genres = genres_data.split('|')
            else:
                movie_genres = []

            # Attribution manuelle du 'movie_id' (à vous de définir comment vous souhaitez les attribuer)
            # Par exemple, vous pouvez utiliser l'index de la boucle actuelle comme 'movie_id'.
            movie_id = index + 1

            # Insérer le film dans Movie_Table
            cursor.execute(f"INSERT INTO {schema_name}.Movie_Table (movie_id, title, release_date, poster_link) VALUES (?, ?, ?, ?)", movie_id, movie_title, release_date, poster_link)


            # Insérer les genres du film dans MovieGenre_Table avec le schéma spécifié
            for genre_name in movie_genres:
                # Récupérer le genre_id pour le genre actuel
                cursor.execute(f"SELECT genre_id FROM {schema_name}.Genre_Table WHERE genre_name = ?", genre_name)
                result = cursor.fetchone()

                if result is not None:
                    genre_id = result[0]

                    # Insérer le movie_id et genre_id dans MovieGenre_Table
                    cursor.execute(f"INSERT INTO {schema_name}.MovieGenre_Table (movie_id, genre_id) VALUES (?, ?)", movie_id, genre_id)
                else:
                    print(f"Genre '{genre_name}' non trouvé dans la table Genre_Table pour le film '{movie_title}'.")

        conn.commit()

        print("Genres insérés avec succès dans la table Genre_Table et liés aux films dans MovieGenre_Table.")

except pyodbc.Error as e:
    print("Erreur lors de la connexion à la base de données SQL Server:", e)

Connexion réussie.
Genres insérés avec succès dans la table Genre_Table et liés aux films dans MovieGenre_Table.
